In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import time
import requests

# Функция для геокодирования адреса с использованием Nominatim API
def geocode_address(address):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': address,
        'format': 'json',
        'limit': 1
    }
    headers = {
        'User-Agent': 'PetFinderApp/1.0 (anton.epub@gmail.com)'
    }
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()  # Проверяем, что запрос успешен
        data = response.json()

        if data:
            location = data[0]
            latitude = float(location['lat'])
            longitude = float(location['lon'])
            return latitude, longitude
        else:
            print("Адрес не найден")
            return None
    except requests.exceptions.HTTPError as e:
        print(f"Ошибка HTTP: {e}")
        print(f"Текст ответа: {response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса: {e}")
        return None
    except ValueError as e:
        print(f"Ошибка парсинга JSON: {e}")
        print(f"Текст ответа: {response.text}")
        return None

# Настройки для Firefox
options = Options()
options.headless = False  # Оставляем окно браузера видимым

# Инициализация драйвера
driver = webdriver.Firefox(options=options)

# URL страницы с объявлениями
base_url = 'https://pet911.ru/catalog'

# Загружаем или создаем DataFrame для хранения ссылок
try:
    df = pd.read_csv('links.csv')
except FileNotFoundError:
    df = pd.DataFrame(columns=['link', 'imgs', 'title', 'description', 'latitude', 'longitude', 'address', 'animal_type', 'status'])

# Удаляем дубликаты ссылок на фотографии в каждой ячейке
df['imgs'] = df['imgs'].apply(lambda x: list(set(eval(x))) if isinstance(x, str) else x)

# Функция для обработки страницы с объявлениями
def process_page(url, df):
    driver.get(url)
    time.sleep(5)  # Ждем, пока страница загрузится

    # Находим все объявления и собираем ссылки
    advertisements = driver.find_elements(By.CLASS_NAME, 'catalog-item')
    links = [ad.find_element(By.CLASS_NAME, 'catalog-item__thumb').get_attribute('href') for ad in advertisements]

    # Фильтруем ссылки, которые уже есть в DataFrame
    new_links = [link for link in links if link not in df['link'].values]

    # Обрабатываем новые ссылки
    for link in new_links:
        print(f"Processing new link: {link}")

        # Переходим на страницу объявления
        driver.get(link)
        time.sleep(3)  # Ждем, пока страница загрузится

        # Находим все картинки в карусели
        image_elements = driver.find_elements(By.CLASS_NAME, 'card-slider__slide')
        imgs = [img.get_attribute('href') for img in image_elements]

        # Удаляем дубликаты ссылок на фотографии
        imgs = list(set(imgs))

        # Находим название объявления
        try:
            title_element = driver.find_element(By.CLASS_NAME, 'card__title')
            title = title_element.find_element(By.TAG_NAME, 'h1').text
        except Exception as e:
            print(f"Ошибка при поиске названия: {e}")
            title = ""

        # Находим описание объявления
        try:
            description_element = driver.find_element(By.CLASS_NAME, 'text.text-lt.card__descr.content')
            description = description_element.find_element(By.TAG_NAME, 'p').text
        except Exception as e:
            print(f"Ошибка при поиске описания: {e}")
            description = ""

        # Находим текстовый адрес
        try:
            address_element = driver.find_element(By.CLASS_NAME, 'card-map__address')
            address = address_element.text
        except Exception as e:
            print(f"Ошибка при поиске адреса: {e}")
            address = None

        # Получаем координаты по текстовому адресу
        latitude, longitude = None, None
        if address:
            coordinates = geocode_address(address)
            if coordinates:
                latitude, longitude = coordinates

        # Извлекаем информацию из breadcrumbs
        try:
            breadcrumbs = driver.find_element(By.CLASS_NAME, 'breadcrumbs')
            breadcrumbs_text = breadcrumbs.text
            if 'Кошки' in breadcrumbs_text:
                animal_type = 'Кошка'
            elif 'Собаки' in breadcrumbs_text:
                animal_type = 'Собака'
            else:
                animal_type = 'Неизвестно'

            if 'Пропавшие' in breadcrumbs_text:
                status = 'Пропала'
            elif 'Найденные' in breadcrumbs_text:
                status = 'Найдена'
            else:
                status = 'Неизвестно'
        except Exception as e:
            print(f"Ошибка при поиске breadcrumbs: {e}")
            animal_type = 'Неизвестно'
            status = 'Неизвестно'

        # Добавляем новую запись в DataFrame
        new_row = pd.DataFrame({
            'link': [link],
            'imgs': [imgs],
            'title': [title],
            'description': [description],
            'latitude': [latitude],
            'longitude': [longitude],
            'address': [address],
            'animal_type': [animal_type],
            'status': [status]
        })
        df = pd.concat([df, new_row], ignore_index=True)

        # Сохраняем DataFrame в CSV файл
        df.to_csv('links.csv', index=False)

    return df

# Функция для обработки записей без координат
def process_missing_coordinates(df):
    missing_coordinates = df[(df['latitude'].isna()) | (df['longitude'].isna())]

    for index, row in missing_coordinates.iterrows():
        link = row['link']
        print(f"Processing link with missing coordinates: {link}")

        # Переходим на страницу объявления
        driver.get(link)
        time.sleep(3)  # Ждем, пока страница загрузится

        # Находим текстовый адрес
        try:
            address_element = driver.find_element(By.CLASS_NAME, 'card-map__address')
            address = address_element.text
        except Exception as e:
            print(f"Ошибка при поиске адреса: {e}")
            address = None

        # Получаем координаты по текстовому адресу
        latitude, longitude = None, None
        if address:
            coordinates = geocode_address(address)
            if coordinates:
                latitude, longitude = coordinates

        # Обновляем запись в DataFrame
        df.loc[index, 'latitude'] = latitude
        df.loc[index, 'longitude'] = longitude

        # Сохраняем DataFrame в CSV файл
        df.to_csv('links.csv', index=False)

    return df

# Проверка на пустые геометки перед запуском
# df = process_missing_coordinates(df)

# Обрабатываем все страницы пагинации
page_number = 220
while True:
    url = f"{base_url}/page-{page_number}"
    print(f"Processing page: {url}")
    df = process_page(url, df)

    # Проверяем, есть ли еще страницы
    try:
        driver.get(url)
        time.sleep(5)  # Ждем, пока страница загрузится
        next_page_link = driver.find_element(By.CSS_SELECTOR, 'a.pagination-arrow[href*="page-"]')
        page_number += 1
    except Exception as e:
        print(f"Ошибка при поиске следующей страницы: {e}")
        break

# Закрываем браузер
driver.quit()


Processing page: https://pet911.ru/catalog/page-220


In [ ]:
driver.quit()

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

# Настройки для Firefox
options = Options()
options.headless = False  # Оставляем окно браузера видимым

# Инициализация драйвера
driver = webdriver.Firefox(options=options)

# URL страницы с объявлением
url = 'https://pet911.ru/krasnogorsk/found/dog/rf925571'

# Открываем страницу
driver.get(url)

# Ждем, пока страница загрузится
time.sleep(5)



In [13]:
map_element = driver.find_element(By.CLASS_NAME, 'leaflet-pane.leaflet-map-pane')


In [14]:
map_element.get_attribute('outerHTML')

'<div class="leaflet-pane leaflet-map-pane" style="transform: translate3d(-406px, 11px, 0px);"><div class="leaflet-pane leaflet-tile-pane"><div class="leaflet-layer " style="z-index: 1; opacity: 1;"><div class="leaflet-tile-container leaflet-zoom-animated" style="z-index: 18; transform: translate3d(0px, 0px, 0px) scale(1);"><img alt="" src="https://core-renderer-tiles.maps.yandex.net/tiles?l=map&amp;z=17&amp;x=79117&amp;y=41040&amp;scale=1&amp;lang=ru_RU" class="leaflet-tile leaflet-tile-loaded" style="width: 256px; height: 256px; transform: translate3d(142px, -81px, 0px); opacity: 1;"><img alt="" src="https://core-renderer-tiles.maps.yandex.net/tiles?l=map&amp;z=17&amp;x=79118&amp;y=41040&amp;scale=1&amp;lang=ru_RU" class="leaflet-tile leaflet-tile-loaded" style="width: 256px; height: 256px; transform: translate3d(398px, -81px, 0px); opacity: 1;"><img alt="" src="https://core-renderer-tiles.maps.yandex.net/tiles?l=map&amp;z=17&amp;x=79117&amp;y=41041&amp;scale=1&amp;lang=ru_RU" class=

In [ ]:
# Находим относительные координаты карты
try:
    # Получаем элемент карты
    map_element = driver.find_element(By.CLASS_NAME, 'leaflet-pane.leaflet-map-pane')
    print("Элемент карты найден")

    print(map_element.html)

    # Получаем стили элемента карты
    style = map_element.get_attribute('style')
    print(f"Стили элемента карты: {style}")

    # Извлекаем значения transform: translate3d
    transform = style.split('transform: ')[1].split(';')[0]
    print(f"Значение transform: {transform}")

    # Разбиваем строку на части
    parts = transform.replace('translate3d(', '').replace('px, ', '').replace('px)', '').split(',')
    print(f"Части transform: {parts}")

    # Проверяем количество частей
    if len(parts) != 3:
        print("Ошибка: неправильное количество частей transform")
    else:
        # Извлекаем значения X, Y, Z
        x, y, z = parts
        print(f"Извлеченные координаты: X={x}, Y={y}, Z={z}")

        # Вывод результата
        print(f"Относительные координаты карты: X={x}, Y={y}, Z={z}")
except Exception as e:
    print(f"Ошибка при извлечении координат: {e}")

# Закрываем браузер
# driver.quit()

In [ ]:
import requests

def geocode_address(address):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': address,
        'format': 'json',
        'limit': 1
    }
    headers = {
        'User-Agent': 'PetFinderApp/1.0 (anton.epub@gmail.com)'
    }
    try:
        print(f"Отправляем запрос на URL: {url} с параметрами: {params}")
        response = requests.get(url, params=params, headers=headers)
        print(f"Статус ответа: {response.status_code}")
        print(f"Текст ответа: {response.text}")
        
        response.raise_for_status()  # Проверяем, что запрос успешен
        data = response.json()
        
        if data:
            location = data[0]
            latitude = float(location['lat'])
            longitude = float(location['lon'])
            return latitude, longitude
        else:
            print("Адрес не найден")
            return None
    except requests.xceptions.RequestException as e:
        print(f"Ошибка запроса: {e}")
        return None
    except ValueError as e:
        print(f"Ошибка парсинга JSON: {e}")
        print(f"Текст ответа: {response.text}")
        return None

# Пример использования
address = "Флотская улица, 56, Москва"
coordinates = geocode_adedress(address)
if coordinates:
    print(f"Координаты: {coordinates[0]}, {coordinates[1]}")

Отправляем запрос на URL: https://nominatim.openstreetmap.org/search с параметрами: {'q': 'Флотская улица, 56, Москва', 'format': 'json', 'limit': 1}
Статус ответа: 200
Текст ответа: [{"place_id":180795758,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright","osm_type":"way","osm_id":28270657,"lat":"55.85395855","lon":"37.50587018283164","class":"building","type":"apartments","place_rank":30,"importance":8.908293278396304e-05,"addresstype":"building","name":"","display_name":"56, Флотская улица, Михалково, Головинский район, Москва, Центральный федеральный округ, 127474, Россия","boundingbox":["55.8531713","55.8547457","37.5055103","37.5062300"]}]
Координаты: 55.85395855, 37.50587018283164
